In [11]:
N = VER = 8
HOR = 1
#Print the State as a reflection of an actual Game Board
def show(state, N=N):
    board = [['.'] * N for _ in range(N)]
    for (c, squares) in state:
        for s in squares:
            x, y = divmod(s, N)
            board[x][y] = c
    # Now print it out
    for row in board:
        print(' '.join(row))
#Returns direction of all objects. Here car is a paramter for any object in the grid be it a red block or wooden block
def direction(car):
    return HOR if car[0]+1 == car[1] else VER

#Determines if a block can move to a new position without colliding with walls or other blocks
def canmove(state, car, next_car):
    usedplaces = [n for a in state for n in a[1] if n not in car and a[0] != "@"]
    return True if len(set(next_car).intersection(set(usedplaces))) == 0 else False

#Returns the new position of a block after it has been moved
def move(state, car, move):
    return tuple(map(sum, zip(car, move)))

#Returns a list of all possible positions that the block can move to from its current position
def auth_pos(state, car, len_car):
    t = []
    mul = direction(car)
    for way in (-1, 1):
        next_car = move(state, car, tuple([way*mul]*len_car))
        while canmove(state, car, next_car):
            t.append(next_car)
            next_car = move(state, next_car, tuple([way*mul]*len_car))
    return t

#Generates all possible next states of the game from its current state 
def next_state(state, N=N):
    t = []
    for car in state:
        if car[0] != '@' and car[0] != '|':
            next_state = auth_pos(state, car[1], len(car[1]))
            if next_state:
                t.append(tuple((car[0],next_state)))
    return tuple(t)

#Maps each possible next state of the game to the move that would achieve that state by returning a dictionary where the keys are the next possible states and values are tuples indicating which block moved and to what position
def successors(state, N=N):
    res = {}
    new_state = next_state(state, N)
    for nstate in new_state:
        for n_pos in nstate[1]:
            res[tuple([c_state if c_state[0] != nstate[0] else (c_state[0], n_pos) for c_state in list(state)])] = tuple((nstate[0], n_pos))
    return res

# Find the shortest path from start state to the goal state where block passes through the goal using a breadth-first search algorithm
#Keeping track of the set of states that we have visited, as long as there are unexplored paths in the frontier, we use the start state to generate successor states.
def shortest_path_search(start, successors, is_goal):
    count = 0
    if is_goal(start):
        return [start]
    explored = set() # set of states we have visited
    frontier = [ [start] ] # ordered list of paths we have blazed
    while frontier:
        count = count + 1
        path = frontier.pop(0) #pop from start of list
        s = path[-1]
        for (state, action) in successors(s).items():
            if state not in explored:
                explored.add(state)
                #if state is unexplored, add it in the explored set and append the action and state to the new path 
                path2 = path + [action, state]
                #After each new successor state is added to the path, check if it is goal state. Hence this is a breadth first search algorithm as for every current state, all possible next states are explored first to check if it is the goal state. For each specific next state, if it is not the goal state, the path is updated with the action and new state, before being appending to the frontier. The algorithm checks all possible moves one move at a time so it will automatically find the shortest path.
                if is_goal(state):
                    print("Total number of paths explored in BFS:", count, "\n")
                    return path2
                else:
                    frontier.append(path2)
    return []

#Solve the puzzle described by the starting position (a tuple of (object, locations) pairs).  Return a path of [state, action, ...]alternating items; an action is a pair (object, new position),such as ('B', (16,17)) to move 'B' to new position @ (16,17)
def bfs_solve_parking_puzzle(start, N=N):
    def is_goal(state):
        car_packed = [n for a in state for n in a[1]]
        return len(set(car_packed))!=len(car_packed)
    #
    sol = shortest_path_search(start, successors, is_goal)
    return sol[1::2]

#Used to create the blocks as tuples with start as the initial position of the block and n as the length of the block
def locs(start, n, incr=1):
    return tuple(n for n in range(start, start + incr * n, incr))


def grid(cars, N=N):
    """Return a tuple of (object, locations) pairs -- the format expected for
    this puzzle.  This function includes a wall pair, ('|', (0, ...)) to 
    indicate there are walls all around the NxN grid, except at the goal 
    location, which is the middle of the right-hand wall; there is a goal
    pair, like ('@', (31,)), to indicate this. The variable 'cars'  is a
    tuple of pairs like ('*', (26, 27)). The return result is a big tuple
    of the 'cars' pairs along with the walls and goal pairs."""
    wall = [w for w in range(0, N + 1)] + \
           [w for w in range(2 * N - 1, N * (N - 1), N)] + \
           [w for w in range(2 * N, N * (N - 1), N)] + \
           [w for w in range(N * (N - 1), N * N)]
    issue = int((N * N - 1) / 2)
    wall.remove(issue)  # + tuple(('|', tuple(sorted(wall))))
    return tuple([('@', (issue,))] + [a for a in cars] + [('|', tuple(sorted(wall)))])


# Here we see the grid and locs functions in use to create the puzzle in our original Lab 7 Report. Blocks such as A or C are vertical blocks, hence the increment is the length of the grid N instead of 1)

puzzle1 = grid((
    ('*', locs(25, 2)),
    ('A', locs(11, 3, N)),
    ('B', locs(13, 2)),
    ('C', locs(22, 3, N)),
    ('D', locs(34, 2, N)),
    ('E', locs(35, 3)),
    ('F', locs(43, 2)),
    ('G', locs(45, 2, N))))


def depth_first_search(start, successors, is_goal):
    count = 0
    if is_goal(start):
        return [start]


    explored = set()  # set of states we have visited
    frontier = [[start]]  # ordered list of paths we have blazed
    while frontier:
        count = count + 1
        path = frontier.pop()  # pop from the end of the list
        s = path[-1] #Check last/current state in the path

        if s not in explored:
            explored.add(s)
            for (state, action) in successors(s).items():
                if state not in explored:
                    new_path = path + [action, state]

                    # Check if a goal state is reached
                    if is_goal(state):
                        print("Total number of paths explored in DFS:", count, "\n")
                        return new_path
                    else:
                        frontier.append(new_path) #depth first search as from start state, all possible successors are added to the frontier. Path at the end of the frontier is popped and last state is explored,the successors of this last state are then added to the end of the frontier, popped and explored. This cycle continues until no goal state is found or the frontier is exhausted

    return []

#Solve the puzzle described by the starting position (a tuple of (object, locations) pairs).  Return a path of [state, action, ...]alternating items; an action is a pair (object, new position),such as ('B', (16,17)) to move 'B' to new position @ (16,17)
def dfs_solve_parking_puzzle(start, N=N):
    """Solve the puzzle described by the starting position (a tuple 
    of (object, locations) pairs).  Return a path of [state, action, ...]
    alternating items; an action is a pair (object, distance_moved),
    such as ('B', 16) to move 'B' two squares down on the N=8 grid."""
    def is_goal(state):
        car_packed = [n for a in state for n in a[1]]
        return len(set(car_packed))!=len(car_packed)

    # show the resolution
    sol = depth_first_search(start, successors, is_goal)
    return sol[1::2]

def path_actions(path):
    "Return a list of actions in this path."
    return path[1::2]


def depth_limited_search(start, successors, is_goal, depth_limit):
    """Find a solution using depth-limited search."""
    count = 0
    if is_goal(start): #Sanity check: Return the start state if it is also the goal state
        return [start]

    explored = set() #Set to keep track of explored states
    #Let frontier represent the set of nodes in the tree that is currently being considered for exploration 
    frontier = [[start]] #Initialize the frontier with the start state as a single path
    while frontier:
        count = count + 1
        path = frontier.pop() #Pop the latest path
        s = path[-1] #Retrieve current state from end of the path

        if len(path) - 1 <= depth_limit:  # Subtract 1 to account for the initial state. #Check if depth of path is within depth limit
         if s not in explored:  #Check that current state has not been explored yet
            explored.add(s)
            for (state, action) in successors(s).items():
                if state not in explored: #Check that the successor state has not been explored
                    new_path = path + [action, state] #Create new path (Extension of current path)
                    if is_goal(state):
                        print("Total number of paths explored in DLS:", count, "\n")
                        return new_path #Return solution path if successor state is goal state
                    else:
                        frontier.append(new_path) #Add new path to end of frontier to be explored again
    return []

def dls_solve_parking_puzzle(start, N=N, depth_limit=400):
    """Solve the puzzle described by the starting position using depth-limited search."""
    def is_goal(state):
        car_packed = [n for a in state for n in a[1]]
        return len(set(car_packed)) != len(car_packed)

    # Solve the puzzle using DLS 
    sol = depth_limited_search(start, successors, is_goal, depth_limit)
    
    #Extract the moves from the solution
    moves = path_actions(sol)
    
    #Return list of moves and number of moves taken to reach the solution
    return moves, len(sol)//2
#** is the red block, each other wooden block represented by 'AA' or 'BB'.... Goal is '@' | are walls of the grid '.' are empty spaces where blocks can move into
show(puzzle1)
print('Breadth First Search Path')
print(bfs_solve_parking_puzzle(puzzle1))
print('Total number of moves:', len(bfs_solve_parking_puzzle(puzzle1)))
print('\n')
print('Depth First Search Path')
print(dfs_solve_parking_puzzle(puzzle1))
print('Total number of moves:', len(dfs_solve_parking_puzzle(puzzle1)))
print('\n')
dls_moves, dls_total_moves = dls_solve_parking_puzzle(puzzle1)
print('Depth Limited Search Path')
print(dls_moves)
print('Total number of moves: ', dls_total_moves)

| | | | | | | |
| . . A . B B |
| . . A . . C |
| * * A . . C @
| . D E E E C |
| . D F F G . |
| . . . . G . |
| | | | | | | |
Breadth First Search Path
Total number of paths explored in BFS: 684 

[('B', (12, 13)), ('C', (14, 22, 30)), ('D', (42, 50)), ('E', (34, 35, 36)), ('G', (21, 29)), ('E', (36, 37, 38)), ('F', (44, 45)), ('A', (35, 43, 51)), ('B', (9, 10)), ('G', (13, 21)), ('*', (28, 29)), ('A', (11, 19, 27)), ('E', (35, 36, 37)), ('C', (38, 46, 54)), ('*', (30, 31))]
Total number of paths explored in BFS: 684 

Total number of moves: 15


Depth First Search Path
Total number of paths explored in DFS: 942 

[('D', (42, 50)), ('E', (33, 34, 35)), ('G', (21, 29)), ('G', (37, 45)), ('G', (29, 37)), ('F', (45, 46)), ('G', (21, 29)), ('F', (44, 45)), ('G', (29, 37)), ('E', (34, 35, 36)), ('G', (21, 29)), ('F', (45, 46)), ('G', (29, 37)), ('F', (43, 44)), ('G', (45, 53)), ('G', (21, 29)), ('G', (37, 45)), ('C', (38, 46, 54)), ('G', (45, 53)), ('G', (21, 29)), ('G', (29, 37)), ('F', 